## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.metrics import pairwise_distances

### Carregando o dataset

In [2]:
df = pd.read_csv('/workspaces/Fuzzy_Clustering/datasets/seeds.csv')
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


### Verificando as classes

In [3]:
df["V8"].unique()

array([1, 2, 3])

### Verificando o nome exato das colunas

In [4]:
df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8'], dtype='object')

### Transformando classes em números

In [5]:
df = df.rename(columns={'V8': 'Class'})
df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
df.head()

/tmp/ipykernel_4230/411401851.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)


,V1,V2,V3,V4,V5,V6,V7,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0


### Verificando a corretude das classes

In [6]:
df["Class"].unique()

array([0, 1, 2])

### Armazenando as classes em uma variável separada

In [7]:
labels = df["Class"].values
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Isolando os dados da classe

In [8]:
df.drop("Class", axis=1, inplace=True)
df.head()

,V1,V2,V3,V4,V5,V6,V7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


### Transformando os dados em uma array

In [9]:
data = df.to_numpy()
data

array([[15.26  , 14.84  ,  0.871 , ...,  3.312 ,  2.221 ,  5.22  ],
       [14.88  , 14.57  ,  0.8811, ...,  3.333 ,  1.018 ,  4.956 ],
       [14.29  , 14.09  ,  0.905 , ...,  3.337 ,  2.699 ,  4.825 ],
       ...,
       [13.2   , 13.66  ,  0.8883, ...,  3.232 ,  8.315 ,  5.056 ],
       [11.84  , 13.21  ,  0.8521, ...,  2.836 ,  3.598 ,  5.044 ],
       [12.3   , 13.34  ,  0.8684, ...,  2.974 ,  5.637 ,  5.063 ]],
      shape=(210, 7))

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik}=1$ para todo $k \in \Omega$.

In [10]:
def init_membership_matrix(n, k):
    membership_matrix = np.random.rand(n, k) # gera uma matriz inicial aleatória com valores entre 0 e 1
    membership_matrix = membership_matrix / membership_matrix.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return membership_matrix

### Inicialização dos medoides

#### 1. Primeiro Medoide
Selecione o primeiro medoide $m_1$ como o ponto com a menor distância total para todos os outros pontos no conjunto de dados $X$, com $n$ amostras:


$m_1 = \arg \min_i \left( \sum_{j=1}^{n} d(x_i, x_j) \right)$


onde $d(x_i, x_j)$ representa a dissimilaridade entre os pontos $x_i$ e $x_j$.

#### 2. Próximos Medoides
Para cada próximo medoide $m_k$, com $k = 2, \dots, c$, encontre o ponto $x$ que maximize a menor distância em relação aos medoides já selecionados. Para cada ponto candidato $x$ (ainda não selecionado como medoide), calcule:


$\text{dist\_mínima}(x) = \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j)$


Então, selecione o ponto $x$ com a maior distância mínima como o próximo medoide:


$m_k = \arg \max_{x \in X \setminus \{m_1, \dots, m_{k-1}\}} \left( \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j) \right)$

In [11]:
def init_medoids(X, c):
    distances = pairwise_distances(X) # calcula todas as distâncias entre os pontos uma vez só

    total_distances = np.sum(distances, axis=1) # primeiro medoide: menor soma de distâncias
    first_medoid_idx = np.argmin(total_distances)

    medoids_indices = [first_medoid_idx] # armazena os índices dos medoides

    for _ in range(1, c): # para os outros medoides
        max_min_dist = -np.inf # armazena a distância
        next_medoid_idx = -1 # armazena o índice do medoide escolhido

        for i in range(len(X)):
            if i in medoids_indices: # se o ponto já for um medoide
                continue

            min_dist_to_medoids = np.min(distances[i, medoids_indices]) # calcula a menor distância deste ponto para qualquer medoide já escolhido

            if min_dist_to_medoids > max_min_dist:
                max_min_dist = min_dist_to_medoids
                next_medoid_idx = i

        medoids_indices.append(next_medoid_idx)

    return X[medoids_indices]

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

#### $u_{ik} = [\sum_{l=1}^c(\frac{d(x_k,v_i)}{d(x_k,v_l)})^{\frac{1}{m-1}}]^{-1}$

In [12]:
def update_membership_matrix(data, medoids, m):
    distance_matrix = pairwise_distances(data, medoids, metric='euclidean') ** 2
    distance_matrix = np.fmax(distance_matrix, np.finfo(np.float64).eps)  # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aqui
    
    inverse_distance_matrix = 1 / distance_matrix
    power = 1 / (m - 1)
    updated_membership_matrix = (inverse_distance_matrix ** power) / np.sum(inverse_distance_matrix ** power, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    
    return updated_membership_matrix

### Atualização dos medoides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

#### $q = \arg \min_{1 \leq j \leq n} \sum_{k=1}^{n} \left( u_{ik} \right)^m \cdot d(x_j, x_k)$

Essa fórmula busca, para cada medoide $m_i$, o ponto $p \in C_i$ que minimiza a soma das distâncias dentro do cluster, garantindo que o novo medoide minimize o custo de distância.

In [13]:
def update_medoids(X, membership_matrix, m=2):
    n, c = X.shape[0], membership_matrix.shape[1]
    distances = pairwise_distances(X, X)
    updated_medoids_indices = []

    for i in range(c):  # para cada cluster
        # custo ponderado total para cada possível medoide j
        costs = np.array([
            np.sum((membership_matrix[:, i] ** m) * distances[j, :])
            for j in range(n)
        ])

        # seleciona o ponto com menor custo como novo medoide
        best_medoid_idx = np.argmin(costs)
        updated_medoids_indices.append(best_medoid_idx)

    return X[updated_medoids_indices]

### Fuzzy C-Medoids (FCMdd)

Etapas:
- Inicialização da matriz de pertinência
- Inicialização dos medoides
- Atualização da matriz de pertinência
- Atualização dos medoides

Critérios de parada:
- Convergência (entre os medoides)
- Número máximo de iterações

In [14]:
def fcmdd(data, k, m=2, max_iter=1000000):
    n = data.shape[0]
    membership_matrix = init_membership_matrix(n, k)
    medoids = init_medoids(data, k)
    for _ in range(max_iter):
        membership_matrix = update_membership_matrix(data, medoids, m)
        new_medoids = update_medoids(data, membership_matrix, m)
        if np.array_equal(medoids, new_medoids): # se os medoides não mudaram, para
            break
        medoids = new_medoids
    return medoids, membership_matrix

### Índice de Rand Ajustado (IRA)

In [15]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [16]:
def monte_carlo_fuzzy_simulation(X, true_labels, k, m=2, num_trials=100):
    results = []
    for trial in range(num_trials):
        medoids, membership_matrix = fcmdd(X, k, m)
        predicted_labels = np.argmax(membership_matrix, axis=1)
        rand_idx = indice_rand(true_labels, predicted_labels)
        results.append(rand_idx)
    
    mean_ari = np.mean(results)
    std_ari = np.std(results)
    return mean_ari, std_ari

### Definição de parâmetros e execução do método

In [17]:
k = 3
num_trials = 100
m = 2
mean_rand_index, std_rand_index = monte_carlo_fuzzy_simulation(data, labels, k, m, num_trials)

print(f"Monte Carlo FCMdd Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {mean_rand_index:.4f}")
print(f"Standard Deviation of Rand Index: {std_rand_index:.4f}")

Monte Carlo FCMdd Clustering Results (100 trials)
Mean Rand Index: 0.6845
Standard Deviation of Rand Index: 0.0000
